# Example usage of [stable-dreamfusion](https://github.com/ashawkey/stable-dreamfusion)
Pure-pytorch version

### Check the machine

In [1]:
! nvidia-smi

Thu Aug  3 22:45:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Setup

In [2]:
#@title install dependencies
! git clone https://github.com/ashawkey/stable-dreamfusion.git

%cd stable-dreamfusion

# install requirements
! pip install -r requirements.txt
! pip install git+https://github.com/NVlabs/nvdiffrast/
! pip install -U diffusers


# install specific version of transformers
!pip install transformers==4.31.0

Cloning into 'stable-dreamfusion'...
remote: Enumerating objects: 1281, done.
remote: Counting objects: 100% (728/728), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 1281 (delta 592), reused 617 (delta 546), pack-reused 553
Receiving objects: 100% (1281/1281), 17.16 MiB | 18.36 MiB/s, done.
Resolving deltas: 100% (778/778), done.
/content/stable-dreamfusion
  Cloning https://github.com/NVlabs/nvdiffrast/ to /tmp/pip-req-build-_6b97g77
  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/nvdiffrast/ /tmp/pip-req-build-_6b97g77
  Resolved https://github.com/NVlabs/nvdiffrast/ to commit c5caf7bdb8a2448acc491a9faa47753972edd380
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-yu9dlwux
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-yu9dlwux
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024

  Cloning https://github.com/NVlabs/nvdiffrast/ to /tmp/pip-req-build-klhn2ama
  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/nvdiffrast/ /tmp/pip-req-build-klhn2ama
  Resolved https://github.com/NVlabs/nvdiffrast/ to commit c5caf7bdb8a2448acc491a9faa47753972edd380
  Preparing metadata (setup.py) ... done


### Training & Testing
* It takes about 0.7s per training step, so the default 5000 training steps take around 1 hour to finish. A larger `Training_iters` usually leads to better results.
* If CUDA OOM, try to decrease `Num_steps`, `Upsample_steps`, and `Training_nerf_resolution`.
* If the NeRF fails to learn anything (empty scene, only background), try to decrease `Lambda_entropy` which regularizes the learned opacity.

In [3]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#@markdown ####**Training Settings:**
Prompt_text = "A tiger" #@param {type: 'string'}
Training_iters = 3000 #@param {type: 'integer'}
Learning_rate = 1e-3 #@param {type: 'number'}
Training_nerf_resolution = 64  #@param {type: 'integer'}
# CUDA_ray = True #@param {type: 'boolean'}
# View_dependent_prompt = True #@param {type: 'boolean'}
# FP16 = True #@param {type: 'boolean'}
Seed = 5 #@param {type: 'integer'}
Lambda_entropy = 1e-4 #@param {type: 'number'}
Num_steps = 64 #@param {type: 'number'}
Upsample_steps = 32 #@param {type: 'number'}
Checkpoint = 'latest' #@param {type: 'string'}

#@markdown ---

#@markdown ####**Output Settings:**
Workspace = "trial_0" #@param{type: 'string'}
Save_mesh = True #@param {type: 'boolean'}

# processings
Prompt_text = "'" + Prompt_text + "'"

In [ ]:
#@title start training
import torch

%run main.py -O2  --text {Prompt_text} --workspace {Workspace} --iters {Training_iters} --lr {Learning_rate} --w {Training_nerf_resolution} --h {Training_nerf_resolution} --seed {Seed} --lambda_entropy {Lambda_entropy} --ckpt {Checkpoint} --save_mesh --num_steps {Num_steps} --upsample_steps {Upsample_steps}

Namespace(file=None, text='A tiger', negative='', O=False, O2=True, test=False, six_views=False, eval_interval=1, test_interval=100, workspace='trial_0', seed='5', image=None, image_config=None, known_view_interval=4, IF=False, guidance=['SD'], guidance_scale=100, save_mesh=True, mcubes_resolution=256, decimate_target=50000.0, dmtet=False, tet_grid_size=128, init_with='', lock_geo=False, perpneg=False, negative_w=-2, front_decay_factor=2, side_decay_factor=10, iters=3000, lr=0.001, ckpt='latest', cuda_ray=False, taichi_ray=False, max_steps=1024, num_steps=64, upsample_steps=32, update_extra_interval=16, max_ray_batch=4096, latent_iter_ratio=0.2, albedo_iter_ratio=0, min_ambient_ratio=0.1, textureless_ratio=0.2, jitter_pose=False, jitter_center=0.2, jitter_target=0.2, jitter_up=0.02, uniform_sphere_rate=0, grad_clip=-1, grad_clip_rgb=-1, bg_radius=1.4, density_activation='exp', density_thresh=10, blob_density=5, blob_radius=0.2, backbone='vanilla', optim='adan', sd_version='2.1', hf_key

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

[INFO] loaded stable diffusion!


[INFO] Cmdline: main.py -O2 --text A tiger --workspace trial_0 --iters 3000 --lr 0.001 --w 64 --h 64 --seed 5 
--lambda_entropy 0.0001 --ckpt latest --save_mesh --num_steps 64 --upsample_steps 32

...

++> Evaluate trial_0 at epoch 30 ...

loss=0.0000 (0.0000): : 100% 8/8 [01:41<00:00, 12.69s/it]


++> Evaluate epoch 30 Finished.

==> Start Test, save results to trial_0/results

 45% 45/100 [09:29<11:36, 12.66s/it]

In [ ]:
#@markdown ####**Testing Settings:**

Workspace_test = "trial_0" #@param{type: 'string'}
# Save_mesh = True #@param {type: 'boolean'}

In [ ]:
#@title testing
import torch
torch.cuda.empty_cache()

%run main.py -O2 --test --workspace {Workspace_test} --save_mesh --max_ray_batch 1024

### Display results
* RGB and Depth video are located at `{Workspace}/results/*.mp4`
* Mesh is under `{Workspace}/mesh/`, you could see three files named `mesh.obj`, `mesh.mtl`, and `albedo.png`.

In [ ]:
#@title display RGB video
import os
import glob
from IPython.display import HTML
from base64 import b64encode

def get_latest_file(path):
  dir_list = glob.glob(path)
  dir_list.sort(key=lambda x: os.path.getmtime(x))
  return dir_list[-1]

def show_video(video_path, video_width = 600):

  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")


rgb_video = get_latest_file(os.path.join(Workspace, 'results', '*_rgb.mp4'))
show_video(rgb_video)